In [14]:
import pandas as pd
#from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
import joblib
import hopsworks


In [15]:

project = hopsworks.login()

fs = project.get_feature_store()

2025-04-20 14:17:35,997 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 14:17:36,237 INFO: Initializing external client
2025-04-20 14:17:36,238 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-20 14:17:37,061 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-20 14:17:37,865 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [16]:
users_fg = fs.get_feature_group(
    name="users",
    version=1
)

events_fg = fs.get_feature_group(
    name="events",
    version=1
)

weather_rank_fg = fs.get_feature_group(
    name="weather_ranking",
    version=1
)

no_weather_rank_fg = fs.get_feature_group(
    name="no_weather_ranking",
    version=1
)

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>

In [17]:
# Select features
selected_features_customers = users_fg.select_all()

fs.get_or_create_feature_view( 
    name='users',
    query=selected_features_customers,
    version=1,
)

In [18]:
# Select features
selected_features_articles = events_fg.select_all()

fs.get_or_create_feature_view(
    name='events',
    query=selected_features_articles,
    version=1,
)

In [19]:
# Select features
features_weather_ranking = weather_rank_fg.select_all()

feature_view_ranking_weather = fs.get_or_create_feature_view(
    name='weather_ranking',
    query=features_weather_ranking,
    labels=["label"],
    version=1,
)

In [20]:
# Select features
features_no_weather_ranking = no_weather_rank_fg.select_all()

feature_view_ranking_no_weather = fs.get_or_create_feature_view(
    name='no_weather_ranking',
    query=features_no_weather_ranking,
    labels=["label"],
    version=1,
)

---

---

---

---

In [21]:
# Get feature views
feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking', version=1)


In [22]:

feature_view_ranking_no_weather = fs.get_feature_view(name='no_weather_ranking', version=1)

In [23]:

# Get training and validation data directly from feature views
weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
    feature_view_ranking_weather.train_test_split(
    test_size=0.1,
    description='Weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.55s) 
2025-04-20 14:18:28,197 WARNING: VersionWarning: Incremented version to `14`.



In [38]:
weather_X_train.head()

,user_id,event_id,interaction_distance_to_event,title,event_type,event_lat,event_lon,event_city,duration,weather_condition,temperature,attendance_rate,event_indoor_capability,user_lat,user_lon,user_city,user_weather_preference,age,user_interests
0,CD639A,PS466T,14.000000,exclusive analyzing secured line business netw...,Business & Networking,40.687006,-73.993215,New York,480,Cloudy,16.6,75.228082,True,40.807395,-74.036095,New York,any,41,tech
1,JI221S,HH055P,16921.000000,synchronized solutionoriented migration commun...,Community & Causes,50.772173,1.711052,Paris,240,Cloudy,16.9,79.345494,False,-33.913905,151.265972,Sydney,outdoor,46,travel
2,NE083K,BD199S,10.584158,cloned valueadded graphical user interface hea...,Health & Wellness,48.848466,2.408878,Paris,240,Clear,18.1,12.692646,False,48.779139,2.310140,Paris,outdoor,49,food
3,KS378O,ZG645S,449.000000,reverseengineered tertiary neuralnet entertain...,Entertainment,42.145335,-78.925101,Toronto,240,Rain,9.1,83.238042,True,40.613727,-73.960229,New York,any,32,cinema tech music sports
4,MB625T,LP926P,7.000000,objectbased local support technology in new york,Technology,40.783807,-73.998889,New York,240,Cloudy,16.0,81.857555,True,40.725007,-74.036360,New York,indoor,34,tech literature


In [24]:

no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = \
    feature_view_ranking_no_weather.train_test_split(
    test_size=0.1,
    description='No-weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.74s) 
2025-04-20 14:18:43,889 WARNING: VersionWarning: Incremented version to `11`.



In [12]:
no_weather_X_train.columns:


Index(['user_id', 'event_id', 'interaction_distance_to_event', 'title',
       'event_type', 'event_lat', 'event_lon', 'event_city', 'duration',
       'attendance_rate', 'event_indoor_capability', 'user_lat', 'user_lon',
       'user_city', 'age', 'user_interests'],
      dtype='object')

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Final version without text fields (title, user_interests)
def train_catboost_without_text_fields(
    train_df, val_df, train_y, val_y
):
    # Drop the text fields if present
    text_columns = ["title", "user_interests"]
    train_df = train_df.drop(columns=[col for col in text_columns if col in train_df.columns])
    val_df = val_df.drop(columns=[col for col in text_columns if col in val_df.columns])

    # Identify categorical features
    cat_features = train_df.select_dtypes(include=["object", "bool"]).columns.tolist()

    # Create CatBoost Pools
    train_pool = Pool(train_df, train_y, cat_features=cat_features)
    val_pool = Pool(val_df, val_y, cat_features=cat_features)

    # Train the model
    model = CatBoostClassifier(
        learning_rate=0.2,
        iterations=100,
        depth=10,
        early_stopping_rounds=5,
        use_best_model=True,
        scale_pos_weight=None, 
        verbose=False
    )


    model.fit(train_pool, eval_set=val_pool)

    # Evaluation
    preds = model.predict(val_pool)
    precision, recall, fscore, _ = precision_recall_fscore_support(val_y, preds, average="binary")
    print("\nClassification Report:")
    print(classification_report(val_y, preds))

    metrics = {
        "precision": precision,
        "recall": recall,
        "fscore": fscore,
    }
    from sklearn.metrics import confusion_matrix
    import numpy as np

    preds = model.scores = model.predict_proba(val_pool)[:, 1] 
    print("Predicted Class Distribution:", np.unique(preds, return_counts=True))

    # print("\nConfusion Matrix:")
    # print(confusion_matrix(val_y, preds))

    return model, metrics, val_pool

"✅ Updated CatBoost training function to exclude title and user_interests for memory safety."


'✅ Updated CatBoost training function to exclude title and user_interests for memory safety.'

In [26]:
# Use this function to train on your weather / no-weather datasets
weather_model, weather_metrics, pool_weather_val = train_catboost_without_text_fields(
    train_df=weather_X_train,
    val_df=weather_X_val,
    train_y=weather_y_train,
    val_y=weather_y_val
)



Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.81      0.75      8838
           1       0.72      0.58      0.64      7527

    accuracy                           0.70     16365
   macro avg       0.71      0.69      0.70     16365
weighted avg       0.71      0.70      0.70     16365

Predicted Class Distribution: (array([0.26349095, 0.27440175, 0.27470047, ..., 0.80501303, 0.80521173,
       0.815168  ]), array([4, 2, 1, ..., 1, 1, 1]))


In [27]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        weather_X_train.columns, 
        weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_distance_to_event': 95.88507441009482,
 'user_lat': 0.5682572734339015,
 'duration': 0.5214805623991308,
 'event_city': 0.5051760410262486,
 'user_city': 0.38683776737670933,
 'title': 0.38084638245406954,
 'attendance_rate': 0.3720852582747443,
 'weather_condition': 0.3307715502825531,
 'user_weather_preference': 0.2969166308206288,
 'temperature': 0.25730246035794035,
 'event_lon': 0.18606856842465705,
 'event_type': 0.1772248463607456,
 'user_lon': 0.13195824869382036,
 'user_id': 0.0,
 'event_id': 0.0,
 'event_lat': 0.0,
 'event_indoor_capability': 0.0}

In [28]:
# Use this function to train on your weather / no-weather datasets
no_weather_model, no_weather_metrics, pool_no_weather_val = train_catboost_without_text_fields(
    train_df=no_weather_X_train,
    val_df=no_weather_X_val,
    train_y=no_weather_y_train,
    val_y=no_weather_y_val
)


Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.81      0.75      8951
           1       0.72      0.58      0.64      7414

    accuracy                           0.71     16365
   macro avg       0.71      0.70      0.70     16365
weighted avg       0.71      0.71      0.70     16365

Predicted Class Distribution: (array([0.25162631, 0.25221187, 0.25478915, ..., 0.83534972, 0.83585689,
       0.83656814]), array([1, 1, 1, ..., 1, 1, 1]))


In [31]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        no_weather_X_train.columns, 
        no_weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_distance_to_event': 90.43820594664992,
 'title': 1.6638176950383345,
 'user_lon': 1.1128959782807077,
 'event_lon': 1.0491738101716028,
 'user_city': 1.028959159330936,
 'event_indoor_capability': 1.0118991412030574,
 'attendance_rate': 0.8997575823223471,
 'duration': 0.6753878567550867,
 'event_city': 0.5604443594303823,
 'event_lat': 0.44797280672878487,
 'user_lat': 0.3903772894026353,
 'event_type': 0.3758839848622131,
 'event_id': 0.34522438982397413,
 'user_id': 0.0}

In [32]:
from sklearn.metrics import roc_auc_score, average_precision_score, ndcg_score, precision_score, recall_score
import numpy as np


def evaluate_ranking_model_proba(model, val_pool, val_y, k_list=[5, 10]):
    """
    Evaluate a CatBoost ranking model using predicted probabilities, not binary class outputs.
    """

    # Predict class probabilities (not class labels)
    proba = model.predict_proba(val_pool)[:, 1]  # Probability for class 1

    results = {
        "AUC": roc_auc_score(val_y, proba),
        "Average Precision (MAP)": average_precision_score(val_y, proba),
    }

    # Convert to numpy arrays
    true_labels = np.array(val_y)
    predicted_scores = np.array(proba)

    # Sort by predicted score
    sorted_indices = np.argsort(predicted_scores)[::-1]
    sorted_true = true_labels[sorted_indices]

    for k in k_list:
        top_k = sorted_true[:k]
        precision_at_k = np.mean(top_k)
        recall_at_k = np.sum(top_k) / np.sum(true_labels)
        ndcg_at_k = ndcg_score(
            y_true=true_labels.reshape(1, -1),
            y_score=predicted_scores.reshape(1, -1),
            k=k
        )



        results[f"Precision@{k}"] = precision_at_k
        results[f"Recall@{k}"] = recall_at_k
        results[f"NDCG@{k}"] = ndcg_at_k

    return results

"✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K."


'✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K.'

In [33]:

# Evaluate weather-aware model
weather_scores = evaluate_ranking_model_proba(
    model=weather_model,
    val_pool=pool_weather_val,
    val_y=weather_y_val
)

# Evaluate no-weather model
no_weather_scores = evaluate_ranking_model_proba(
    model=no_weather_model,
    val_pool=pool_no_weather_val,
    val_y=no_weather_y_val
)

# Compare results
print("Weather Model Scores:")
for k, v in weather_scores.items():
    print(f"{k}: {v:.4f}")

print("\nNo-Weather Model Scores:")
for k, v in no_weather_scores.items():
    print(f"{k}: {v:.4f}")


Weather Model Scores:
AUC: 0.7085
Average Precision (MAP): 0.6776
Precision@5: 0.6000
Recall@5: 0.0004
NDCG@5: 0.6992
Precision@10: 0.7000
Recall@10: 0.0009
NDCG@10: 0.7334

No-Weather Model Scores:
AUC: 0.7043
Average Precision (MAP): 0.6690
Precision@5: 1.0000
Recall@5: 0.0007
NDCG@5: 1.0000
Precision@10: 0.9000
Recall@10: 0.0012
NDCG@10: 0.9266


In [34]:
# Connect to Hopsworks Model Registry
model_registry = project.get_model_registry()

In [35]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib

def upload_catboost_model_to_hopsworks(
    model, model_name: str, model_description: str,
    model_registry, X_train, y_train, metrics: dict,
    local_path
):
    # Save the model locally
    joblib.dump(model, local_path)

    # Create input/output schema
    input_schema = Schema(X_train)
    output_schema = Schema(y_train)
    model_schema = ModelSchema(input_schema, output_schema)

    # Sample input for UI representation
    input_example = X_train.sample().to_dict("records")

    # Create and save model in Hopsworks
    hopsworks_model = model_registry.python.create_model(
        name=model_name,
        metrics=metrics,
        model_schema=model_schema,
        input_example=input_example,
        description=model_description,
    )

    hopsworks_model.save(local_path)

    print(f"✅ Model '{model_name}' successfully uploaded to Hopsworks.")
    return hopsworks_model

"✅ Model upload function is ready for Hopsworks. Pass your CatBoost model, train data, metrics, and registry reference."


'✅ Model upload function is ready for Hopsworks. Pass your CatBoost model, train data, metrics, and registry reference.'

In [36]:
upload_catboost_model_to_hopsworks(
    model=weather_model,
    model_name="weather_ranking_model",
    model_description="Ranking model trained with weather features",
    model_registry=project.get_model_registry(),
    X_train=weather_X_train, 
    y_train=weather_y_train,
    metrics=weather_metrics,
    local_path="weather_ranking_model.pkl"
)


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/262968 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/598 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1586 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/weather_ranking_model/2
✅ Model 'weather_ranking_model' successfully uploaded to Hopsworks.


Model(name: 'weather_ranking_model', version: 2)

In [37]:
upload_catboost_model_to_hopsworks(
    model=no_weather_model,
    model_name="no_weather_ranking_model",
    model_description="Ranking model trained without weather features",
    model_registry=project.get_model_registry(),
    X_train=no_weather_X_train, 
    y_train=no_weather_y_train,
    metrics=no_weather_metrics,
    local_path="no_weather_ranking_model.pkl"
)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/901551 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/505 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1348 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/no_weather_ranking_model/2
✅ Model 'no_weather_ranking_model' successfully uploaded to Hopsworks.


Model(name: 'no_weather_ranking_model', version: 2)